<a href="https://colab.research.google.com/github/Sohanngolla/Fake-Profile-Detection-across-social-media-/blob/main/Human_detection_in_disater_management_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================
# SIMPLE HUMAN DETECTION (PEOPLE COUNTER)
# ========================================

print("📦 Installing dependencies...")
!pip install -q pillow opencv-python-headless ultralytics pillow-heif
print("✅ Ready!\n")

import os
import cv2
import numpy as np
from PIL import Image
from google.colab import files
from ultralytics import YOLO
import pillow_heif

# Register HEIF opener for iPhone photos
pillow_heif.register_heif_opener()

# 1. LOAD MODEL (Standard Detection Only)
# We use 'yolov8n.pt' which is trained on COCO (80 classes)
print("🤖 Loading YOLOv8 Nano model...")
model = YOLO('yolov8n.pt')
print("✅ Model loaded!\n")

def convert_to_jpg(image_path):
    """Sanitize image format"""
    try:
        if not os.path.exists(image_path): return None
        img = Image.open(image_path)
        if img.mode != 'RGB': img = img.convert('RGB')
        jpg_path = image_path.rsplit('.', 1)[0] + '_fixed.jpg'
        img.save(jpg_path, 'JPEG', quality=95)
        return jpg_path
    except Exception as e:
        print(f"Error: {e}")
        return None

def draw_box(image, bbox, confidence, person_id):
    """Draw just the box and label"""
    x1, y1, x2, y2 = map(int, bbox[:4])

    # Draw Rectangle (Green)
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # Label
    label = f"Person {person_id} ({confidence:.0%})"

    # Text settings
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    thickness = 2
    (w, h), _ = cv2.getTextSize(label, font, font_scale, thickness)

    # Draw label background (Black filled box for readability)
    cv2.rectangle(image, (x1, y1 - 25), (x1 + w, y1), (0, 0, 0), -1)

    # Draw text (White)
    cv2.putText(image, label, (x1, y1 - 5), font, font_scale, (255, 255, 255), thickness)

    return image

# MAIN EXECUTION
print("📤 UPLOAD IMAGE")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    clean_path = convert_to_jpg(filename)

    if clean_path:
        # Load image into OpenCV
        img = cv2.imread(clean_path)

        # 2. RUN INFERENCE
        # classes=0 tells YOLO to ONLY look for persons (Class 0 in COCO)
        # conf=0.4 means we need 40% confidence to accept a detection
        results = model(clean_path, classes=0, conf=0.4, verbose=False)

        # 3. PROCESS RESULTS
        result = results[0]
        boxes = result.boxes
        num_people = len(boxes)

        print(f"\n✅ DETECTED: {num_people} people.")

        # Draw boxes
        for i, box in enumerate(boxes):
            coords = box.xyxy[0].cpu().numpy() # Get x1, y1, x2, y2
            conf = float(box.conf[0].cpu().numpy()) # Get confidence
            img = draw_box(img, coords, conf, i+1)

        # Save and Show
        output_path = "detected_people.jpg"
        cv2.imwrite(output_path, img)

        from IPython.display import display
        display(Image.open(output_path))
        files.download(output_path)